# KI Projekt Verkehrsszenario _ Verhaltensprädiktion

### Importieren der benötigten Bibliotheken

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

### Importieren des Datensatzes

In [ ]:
df_all = pd.read_csv("SIM_002.csv", delimiter=";")
df_input = pd.read_csv("Input.csv", delimiter=";")
df_all.dtypes


### Konvertieren der Objekt-Zahlenwerte zu Float Zahlenwerten
Nachdem auffällt, dass v, v_left und v_front als object eingelesen werden, müssen diese Spalten hier noch konvertiert werden.

In [ ]:
df_all = df_all.replace(',','.', regex=True)
df_all['v'] = pd.to_numeric(df_all['v'])
df_all['v_left'] = pd.to_numeric(df_all['v_left'])
df_all['v_front'] = pd.to_numeric(df_all['v_front'])

df_input = df_input.replace(',','.', regex=True)
df_input['v'] = pd.to_numeric(df_input['v'])
df_input['v_left'] = pd.to_numeric(df_input['v_left'])
df_input['v_front'] = pd.to_numeric(df_input['v_front'])


df_all

### Umrechnen der Geschwindigkeiten von m/s auf Km/h
Dies wird gemacht, um einheitliche Werte zu haben. Maximalgeschwindigkeit und die Geschwindigkeiten der Autos unterscheiden sich in der Einheit.

In [ ]:
df_all['v'] = df_all['v'] * 3.6
df_all['v_left'] = df_all['v_left'] * 3.6
df_all['v_front'] = df_all['v_front'] * 3.6

df_input['v'] = df_input['v'] * 3.6
df_input['v_left'] = df_input['v_left'] * 3.6
df_input['v_front'] = df_input['v_front'] * 3.6

df_all

#### Bereinigen des Datensets wo notwendig

Es ist keinerlei Datensatz aufgefallen, welcher entfernt werden muss, da er ungültige Werte (bspw. leere Felder oä) enthält.

### Encoding der nicht-nummerischen Werte
Die folgenden features sind vom typ object und benötigen deshalb ein label encoding, damit später besser mit ihnen gearbeitet werden kann.
Das label encoding wird mit der Bibliothek scikit learn durchgeführt.
#### type-left:
* car -> 0
* motorcycle -> 1
* sportscar -> 2
* truck -> 3


In [ ]:

print(df_all['type_left'].value_counts())

# Do the label encoding with sklearn
type_left_encoder = preprocessing.LabelEncoder().fit(df_all["type_left"])
print(dict(zip(type_left_encoder.classes_, type_left_encoder.transform(type_left_encoder.classes_))))
df_all["type_left"] = type_left_encoder.transform(df_all["type_left"])
df_input["type_left"] = type_left_encoder.transform(df_input["type_left"])

#### type-front:
* car -> 0
* motorcycle -> 1
* sportscar -> 2
* truck -> 3

In [ ]:
print(df_all['type_front'].value_counts())

# Do the label encoding with sklearn
type_front_encoder = preprocessing.LabelEncoder().fit(df_all["type_front"])
print(dict(zip(type_front_encoder.classes_, type_front_encoder.transform(type_front_encoder.classes_))))
df_all["type_front"] = type_front_encoder.transform(df_all["type_front"])
df_input["type_front"] = type_front_encoder.transform(df_input["type_front"])

#### slope_street:
* ascending -> 0
* decending -> 1
* flat -> 2

In [ ]:
print(df_all['slope_street'].value_counts())

# Do the label encoding with sklearn
slope_street_encoder = preprocessing.LabelEncoder().fit(df_all["slope_street"])
print(dict(zip(slope_street_encoder.classes_, slope_street_encoder.transform(slope_street_encoder.classes_))))
df_all["slope_street"] = slope_street_encoder.transform(df_all["slope_street"])
df_input["slope_street"] = slope_street_encoder.transform(df_input["slope_street"])

#### street_type:
* autobahn -> 0
* country_road (separated) -> 1

In [ ]:
print(df_all['street_type'].value_counts())

# Do the label encoding with sklearn
street_type_encoder = preprocessing.LabelEncoder().fit(df_all["street_type"])
print(dict(zip(street_type_encoder.classes_, street_type_encoder.transform(street_type_encoder.classes_))))
df_all["street_type"] = street_type_encoder.transform(df_all["street_type"])
df_input["street_type"] = street_type_encoder.transform(df_input["street_type"])

#### time:
* dawn -> 0
* day -> 1
* dusk -> 2
* night -> 3

In [ ]:
print(df_all['time'].value_counts())

# Do the label encoding with sklearn
time_encoder = preprocessing.LabelEncoder().fit(df_all["time"])
print(dict(zip(time_encoder.classes_, time_encoder.transform(time_encoder.classes_))))
df_all["time"] = time_encoder.transform(df_all["time"])
df_input["time"] = time_encoder.transform(df_input["time"])

#### weather:
* dry -> 0
* fog -> 1
* rain -> 2
* snow_ice -> 3

In [ ]:
print(df_all['weather'].value_counts())

# Do the label encoding with sklearn
weather_encoder = preprocessing.LabelEncoder().fit(df_all["weather"])
print(dict(zip(weather_encoder.classes_, weather_encoder.transform(weather_encoder.classes_))))
df_all["weather"] = weather_encoder.transform(df_all["weather"])
df_input["weather"] = weather_encoder.transform(df_input["weather"])

#### type_vehicle:
* car -> 0
* motorcycle -> 1
* sportscar -> 2
* truck -> 3

In [ ]:
print(df_all['type_vehicle'].value_counts())

# Do the label encoding with sklearn
type_vehicle_encoder = preprocessing.LabelEncoder().fit(df_all["type_vehicle"])
print(dict(zip(type_vehicle_encoder.classes_, type_vehicle_encoder.transform(type_vehicle_encoder.classes_))))
df_all["type_vehicle"] = type_vehicle_encoder.transform(df_all["type_vehicle"])
df_input["type_vehicle"] = type_vehicle_encoder.transform(df_input["type_vehicle"])

#### action:
One Hot Encoding für alle action Werte um sie einzeln betrachten zu können für die Korrelationen. Dies ist eine Methode, um mit Attributen zu arbeiten, welche keine Nominalskala haben, die Korrelationen trotzdem interessant ist. Bei anderen Attributen wurde darauf verzichtet, da die Korrelation hier weniger interessant ist. Außerdem wird das encoding nur in eine Kopie des eigentlichen Datenframes geschrieben, um es später nicht für die Vorhersage zu nutzen, sondern nur für die Korrelationsheatmap.

In [ ]:
df_analysis = df_all.copy()
actions = ['accelerated_lane_change', 'continue', 'decelerate', 'lane_change']
for x in actions:
    df_analysis["{}".format(x)] = np.where(df_analysis.action == x, 1, 0)
    
df_analysis

In [ ]:
plt.figure(figsize=(14, 7))
sns.heatmap(data=df_analysis.corr(numeric_only=True), vmin=-1, vmax=1, annot=True, cmap="magma").set_title('Korrelation Heatmap aller Attribute', fontdict={'fontsize': 14}, pad=12)

### Evaluation der Korrelationsheatmap
Da die Geschwindigkeit ein zentraler Faktor in der Überholungs-Überlegung ist, wird diese zuerst betrachtet. Sie korreliert in der Heatmap sehr stark mit v_left, ``v_front`` und ``d_front``. Außerdem ist auffällig, dass eine negative Korrelation mit ``type_front`` und ``type_vehicle`` besteht. Diese kann aber wegen nicht vorhandener Rangordnung in den Attributen allerdings nicht in Betracht gezogen werden. 

type_front korreliert zwar mit `v`, `v_left`, `v_front` und `d_front` negativ, allerdings ist auch hier keine Nominalskala vorhanden.

Der wichtigste Faktor für das Training ist ``action``. Dieser Wert teilt sich hier in vier verschiedene Attribute auf. Es ist eine leichte Korrelation zwischen der ``action`` 'continue' und ``v_front`` zu erkennen. Vermutlich wird also eher 'continue' gewählt, wenn der Abstand zum vorrausfahrenden Auto größer ist. 
``v_front`` korreliert außerdem auch leicht negativ mit den anderen ``action``s, dabei am stärksten mit ``lane_change``. Also je kleiner der Abstand zum vorrausfahrenden Auto ist, desto eher wird abgebremst, Spur gewechselt oder beschleunigt die Spur gewechselt.

Außerdem ist zwischen ``accelerated_lane_change`` und ``d_left`` eine leichte Korrelation zu erkennen. Je weiter das Auto auf der linken Spur also entfernt ist, desto größer ist die Wahrscheinlichkeit für einen beschleunigten Spurwechsel. 


### Geschwindigkeit Differenz Berechnung zur action-Bewertung
Dazu werden zum einem die Differnz der Geschwindigkeiten von 'v' und 'v_front' berechnet, um einschätzen zu können, ob die Geschwindigkeit beibehalten werden soll, ob verzögert oder überholt soll. 

Außerdem werden 'v' und 'v_left' verglichen, um bewerten zu können, ob und wie (mit oder ohne Beschleunigung) eine Überholung stattfinden soll.

In [ ]:
for i in range(1, len(df_all) +1):
    df_all['delta_vFront'] = df_all['v'] - df_all['v_front']
    df_all['delta_vLeft'] = df_all['v'] - df_all['v_left']
    df_all['delta_speed_limit'] = df_all['speed_limit(km/h)'] - df_all['v']
    df_input['delta_vFront'] = df_input['v'] - df_input['v_front']
    df_input['delta_vLeft'] = df_input['v'] - df_input['v_left']
    df_input['delta_speed_limit'] = df_input['speed_limit(km/h)'] - df_input['v']

df_all

### Trainings- und Testdaten aufteilen

Die Daten werden hier aufgeteilt. Um später eine Cross-Validierung möglich zu machen, werden je nach Eingabe verschiedene Intervalle gewählt. Diese sind entweder mit den Testdaten bei 0 - 100, 100 - 200 etc.

In [ ]:
# interval got to be between 0 and 4
def get_test_and_train_data(interval):
    lower_border = interval * 100
    upper_border = lower_border + 100
    if lower_border != 0:
        train_data = df_all.loc[:lower_border]
        train_data = train_data.append(df_all.loc[(upper_border+1):])
        test_data = df_all.loc[(lower_border + 1):upper_border]
    else:
        train_data = df_all.loc[(upper_border):]
        test_data = df_all.loc[(lower_border):upper_border-1]
    return train_data, test_data

## Prüfung auf Gleichheit 
In dieser Funktion werden die Testdaten auf Gleichheit überprüft. Das bedeutet, dass die Eingabe mit allen Trainingsdaten verglichen wird und auf eine Überseinstimmung untersucht wird. Sobald Trainingsdaten mit den exakt gleichen Werten gefunden wurden, wird die gleiche 'action' für die Testdaten übernommen.

In [ ]:
def check_for_same(input, train_data):
    for index, row in train_data.iterrows():
        if input['v'] == row['v'] and input['v_left'] == row['v_left'] and input['v_front'] == row['v_front'] and input['d_left'] == row['d_left'] and input['d_front'] == row['d_front'] and input['type_left'] == row['type_left'] and input['type_front'] == row['type_front'] and input['radius_curve(m)'] == row['radius_curve(m)'] and input['slope_street'] == row['slope_street'] and input['street_type'] == row['street_type'] and input['time'] == row['time'] and input['weather'] == row['weather'] and input['type_vehicle'] == row['type_vehicle'] and input['speed_limit(km/h)'] == row['speed_limit(km/h)'] and input['delta_vFront'] == row['delta_vFront'] and input['delta_vLeft'] == row['delta_vLeft']:
            return True, row
    return False, 0

## Prüfung auf Ähnlichkeit

Nachdem auf Gleichheit geprüft wurde und der Fall auftritt, dass keine genau gleichen Trainingsdaten gefunden wurden, wird auf Ähnlichkeit geprüft. 
Das heißt, dass alle Attribute verglichen werden und nach dem ähnlichsten Fall in den Trainingsdaten gesucht wird. 

Um am Schluss eine Formel für das Ähnlichkeitsmaß aufstellen zu können, müssen die verschiedenen Attribute alle einen Wert bekommen, der in dem Bereich von 0 und 1 liegt. 

In [ ]:
# Similarity of speed with a value between 0 and 1 
# default and to_compare are the input and training data
# both, default and to_compare, contains positive values 
def similarity_v(default, to_compare):
    return 1 - ((abs(default - to_compare))/(abs(default) + abs(to_compare)))

def similarity_v_left(default, to_compare):
    return similarity_v(default, to_compare)

def similarity_v_front(default, to_compare):
    return similarity_v(default, to_compare)

# d_left contains negative values as well and therefore a different actions are necessary
def similarity_d_left(default, to_compare):
    if default == 0 and to_compare == 0: # both 0 
        return 1
    elif default >= 0 and to_compare >= 0: # both positive 
        return similarity_v(default, to_compare)
    elif default < 0 and to_compare < 0: # both negative 
        return similarity_v(default, to_compare)
    # if one number is negative, take its abs value and keep the distance between the numbers
    elif default < 0 and to_compare >= 0:
        default = abs(default)
        to_compare += 2 * default
        return similarity_v(default, to_compare)
    elif default >= 0 and to_compare < 0:
        to_compare = abs(to_compare)
        default += 2 * to_compare
        return similarity_v(default, to_compare)
    else:
        return 0

# similarity of positive numeric values: 
def similarity_d_front(default, to_compare):
    return similarity_v(default, to_compare)

def similarity_radius_curve(default, to_compare):
    return similarity_v(default, to_compare)

def similarity_speed_limit(default, to_compare):
    return similarity_v(default, to_compare)

def similarity_delta_vFront(default, to_compare):
    return similarity_v(default, to_compare)

# similarity of mixed numeric values -> therefore similarity_d_left is necessary
def similarity_delta_vLeft(default, to_compare):
    return similarity_d_left(default, to_compare)

def similarity_delta_speed_limit(default, to_compare):
    return similarity_d_left(default, to_compare)

### Distanzberechnung der type_left, type_front und type_vehicle Fahrzeuge
0: motorcycle and truck<br>
0.1: sportscar and truck<br>
0.3: car and motorcycle<br>
0.5: sportscar and motorcycle<br>
0.6: car and truck<br>
0.8: car and sportscar<br>
1: same<br>

In [ ]:
# 0 = car, 1 = motorcycle, 2 = sportscar, 3 = truck
def similarity_type_left(default, to_compare):
    if (default == 1 and to_compare == 3) or (default == 3 and to_compare == 1): # motorcycle and truck
        return 0.0
    if (default == 2 and to_compare == 3) or (default == 3 and to_compare == 2): # sportscar and truck
        return 0.1
    if (default == 0 and to_compare == 1) or (default == 1 and to_compare == 0): # car and motorcycle
        return 0.3
    if (default == 2 and to_compare == 1) or (default == 1 and to_compare == 2): # sportscar and motorcycle
        return 0.5
    if (default == 0 and to_compare == 3) or (default == 3 and to_compare == 0): # car and truck
        return 0.6
    if (default == 0 and to_compare == 2) or (default == 2 and to_compare == 0): # car and sportscar
        return 0.8    
    if (default == to_compare): # same
        return 1.0

# same vehicles and encodings 
def similarity_type_front(default, to_compare):
    return similarity_type_left(default, to_compare)

# same vehicles and encodings 
def similarity_type_vehicle(default, to_compare):
    return similarity_type_left(default, to_compare)

### Distanzberechnung der Routenbeschaffenheit


In [ ]:
# ascending = 0, descending = 1, flat = 2
def similarity_slope_street(default, to_compare):
    if (default == 1 and to_compare == 2) or (default == 2 and to_compare == 1): # descending and flat
        return 0.5
    elif (default == 0 and to_compare == 2) or (default == 2 and to_compare == 0): # ascending and flat
        return 0.5
    elif default == to_compare:
        return 1
    else:
        return 0   

### Distanzberechnung der Straßenart 

In [ ]:
# to different types of streets 
# same -> 1 
# different (autobahn and country_road) -> 0
def similarity_street_type(default, to_compare):
    if default == to_compare:
        return 1
    else:
        return 0

### Distanzberechnung für das Wetter
0.0: dry and snow ice<br>
0.1: fog and snow ice<br>
0.1: rain and snow ice<br>
0.5: dry and fog<br>
0.6: fog and rain<br>
0.8: dry and rain<br>
1: same<br>

In [ ]:
# dry = 0, fog = 1, rain = 2, snow_ice = 3 
def similarity_weather(default, to_compare):
    if (default == 0 and to_compare == 3) or (default == 3 and to_compare == 0): # dry and snow-ice 
        return 0
    if (default == 1 and to_compare == 3) or (default == 3 and to_compare == 1): # fog and snow-ice 
        return 0.1
    if (default == 2 and to_compare == 3) or (default == 3 and to_compare == 2): # rain and snow-ice 
        return 0.1
    if (default == 0 and to_compare == 1) or (default == 1 and to_compare == 0): # dry and fog 
        return 0.5
    if (default == 2 and to_compare == 1) or (default == 1 and to_compare == 2): # fog and rain
        return 0.6
    if (default == 2 and to_compare == 0) or (default == 0 and to_compare == 2): # dry and rain
        return 0.8
    if (default == to_compare): # same
        return 1.0

### Distanzberechnung für die verschiedenen Tageszeiten
0: day and night<br>
0.4: dusk and night<br>
0.5: dawn and night<br>
0.5: day and dawn<br>
0.6: day and dusk<br>
0.8: dawn and dusk<br>
1: same<br>

In [ ]:
# dawn = 0, day = 1, dusk = 2, night = 3
def similarity_time(default, to_compare):
    if (default == 1 and to_compare == 3) or (default == 3 and to_compare == 1): # day and night
        return 0.0
    if (default == 2 and to_compare == 3) or (default == 3 and to_compare == 2): # dusk and night
        return 0.4
    if (default == 0 and to_compare == 3) or (default == 3 and to_compare == 0): # dawn and night
        return 0.5
    if (default == 0 and to_compare == 1) or (default == 1 and to_compare == 0): # day and dawn
        return 0.5
    if (default == 1 and to_compare == 2) or (default == 2 and to_compare == 1): # day and dusk
        return 0.6
    if (default == 0 and to_compare == 2) or (default == 2 and to_compare == 0): # dawn and dusk
        return 0.8    
    if (default == to_compare): # same
        return 1.0

### Das ähnlichste Element finden
Zuerst werden die Gewichtungen für jedes einzelne Attribut festgelegt. Anschließend wird die Ähnlichkeit (similarity) berechnet. Dies funktioniert über Gewichtung * Ähnlichkeit, beide Werte zwischen null und eins. Anschließend werden die gewichteten Ähnlichkeiten aller Attribute aufsummiert und mit dem bisher ähnlichsten Fall verglichen. Wenn neue Fall ein höheres Ähnlichkeitsmaß erreicht, wird er selbst zum bisher höchsten gefundenen. Am Ende der Schleife wird die in dem Moment ähnlichste Situation zurückgegeben.  

In [ ]:
# try to find the most similar case 
def check_for_similar(input, train_data):
    # weights setting:
    weight_v = 1
    weight_v_left = 1
    weight_v_front = 0.4
    weight_d_left = 1
    weight_d_front = 1
    weight_type_left = 0.1
    weight_type_front = 0.1
    weight_radius_curve = 0.7
    weight_slope_street = 0.1
    weight_street_type = 0.1
    weight_time = 0.1
    weight_weather = 0.8
    weight_type_vehicle = 0.1
    weight_speed_limit = 1
    weight_delta_vFront = 0.8
    weight_delta_vLeft = 0.4
    weight_delta_speed_limit = 0.2

    max_sim = 0
    max_row = 0

    # comparing attributes from training_data and input 
    for i in range(len(train_data)):
        row = train_data.iloc[i]
        # similarity calculation 
        similarity = weight_v * similarity_v(input['v'], row['v'])\
        + weight_v_left * similarity_v_left(input['v_left'], row['v_left']) \
        + weight_v_front * similarity_v_front(input['v_front'], row['v_front']) \
        + weight_d_left * similarity_d_left(input['d_left'], row['d_left'])  \
        + weight_d_front * similarity_d_front(input['d_front'], row['d_front'])  \
        + weight_type_left * similarity_type_left(input['type_left'], row['type_left'])  \
        + weight_type_front * similarity_type_front(input['type_front'], row['type_front']) \
        + weight_radius_curve * similarity_radius_curve(input['radius_curve(m)'],row['radius_curve(m)']) \
        + weight_slope_street * similarity_slope_street(input['slope_street'], row['slope_street']) \
        + weight_street_type * similarity_street_type(input['street_type'], row['street_type']) \
        + weight_time * similarity_time(input['time'], row['time']) \
        + weight_weather * similarity_weather(input['weather'], row['weather']) \
        + weight_type_vehicle * similarity_type_vehicle(input['type_vehicle'], row['type_vehicle']) \
        + weight_speed_limit * similarity_speed_limit(input['speed_limit(km/h)'], row['speed_limit(km/h)']) \
        + weight_delta_vFront * similarity_delta_vFront(input['delta_vFront'], row['delta_vFront']) \
        + weight_delta_vLeft *  similarity_delta_vLeft(input['delta_vLeft'], row['delta_vLeft']) \
        + weight_delta_speed_limit * similarity_delta_speed_limit(input['delta_vFront'], row['delta_vFront'])
        if similarity > max_sim: # if current similarity bigger than the previous one, use the current row 
            max_row = row
            max_sim = similarity
        
     # use highest similarity and the corresponding action for return
    return max_row

### Vorhersage der Action
Es wird zuerst geschaut, ob ein gleicher Fall in den Daten gefunden wird. Falls dies der Fall ist, wird sich gleich verhalten wie in dem gefundenen Fall. Wenn kein identischer Fall vorhanden ist, wird mit ``check_for_similar`` nach dem ähnlichsten Fall gesucht. Es wird sich dann so verhalten, wie in dem gefundenen Fall. Für bessere Nachvollziehbarkeit kann die Flag ``print_output`` genutzt werden.

In [ ]:
def do_prediction(input, train_data, print_output = False):
    # first check for equal data 
    success, prediction = check_for_same(input, train_data)
    # if no matching data was found -> check for the most similar case 
    if not success:
        prediction = check_for_similar(input, train_data)
    # debug mode
    if print_output:
        print('Input: \n', input)
        print("\n-Prediction-\n")
        print('Prediction: \n', prediction)
    return prediction['action']

# input prediction:
print('\n\nPredicted action for input case: ', do_prediction(df_input.iloc[0], df_all, print_output=True))


### Berechnung der Accuracy

Dabei wird die Genauigkeit des Modells berechnet, indem alle fünf Durchläufe mit den unterschiedlichen Test- und Trainingsdaten aufsummiert werden und dann durch die Länge, also die Anzahl der Durchläufe, geteilt wird. 

Außerdem wird nach der Suche einer Eingabe, die Eingabe und die Lösung des ähnlichen Falls aus den Trainingsdaten weiter verwendet. Dafür werden die Daten dem Trainingsdatensatz hinzugefügt. 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

def evaluate_accuracy(add_test_line = False):
    accuracy_sum = []
    for j in range(0,5):
        train_data, test_data = get_test_and_train_data(j)
        accuracy = 0
        for i in range(len(test_data)):
            row = test_data.iloc[i]
            prediction = do_prediction(row, train_data)
            if prediction == row['action']:
                accuracy += 1
            # Retain phase
            if add_test_line:
                train_data = train_data.append(row)
        accuracy = accuracy / len(test_data)
        print('accuracy ', str(j+1), ' is: ', accuracy, ' / ', len(test_data))
        accuracy_sum.append(accuracy)
    accuracy_average = np.sum(accuracy_sum)/len(accuracy_sum)
    print('accuracy total: ', accuracy_average)
evaluate_accuracy(False)
